##WanDB 사용체험
Validation data를 포함하여 Fine-tuning 해보기
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W7_1_Basic/W7_1_Basic.ipynb)

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn

In [5]:
import os
import sys
import math
import torch
import wandb
import logging
import datasets
import argparse
import evaluate
import transformers

from typing import Optional
from itertools import chain
from dataclasses import dataclass, field

In [6]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DebertaConfig, AutoModelForSequenceClassification

In [7]:
from datasets import load_dataset
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator
)
from transformers.trainer_utils import get_last_checkpoint


In [8]:
#device setting
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.backends.cuda.is_built():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [9]:
wandb.init(project='Hanghae99')
wandb.run.name = 'gpt-finetuning'

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kimguangdong (kimguangdong-js) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [10]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [11]:
@dataclass
class Arguments:
    model_name_or_path: Optional[str] = field(default=None)
    torch_dtype: Optional[str] = field(default=None, metadata={'choices': ['auto', 'bfloat16', 'float16', 'float32']})

    dataset_name: Optional[str] = field(default=None)
    dataset_config_name: Optional[str] = field(default=None)
    block_size: int = field(default=1024)
    num_workers: Optional[int] = field(default=None)

필드 설명:
model_name_or_path:
사용할 사전 학습 모델의 경로나 이름 (e.g., "bert-base-uncased", "facebook/opt-1.3b").None이면 기본값을 사용하거나 별도로 설정해야 함.

torch_dtype:
모델을 어떤 데이터 타입으로 로드할지 결정 (auto, bfloat16, float16, float32 중 선택 가능). bfloat16이나 float16을 선택하면 VRAM 사용량이 줄어듦.

dataset_name:
Fine-tuning에 사용할 Hugging Face 데이터셋의 이름 (e.g., "wikitext", "glue").

dataset_config_name: 데이터셋의 특정 설정 (예: "wikitext-2-v1").
block_size: 입력 시퀀스 길이 (기본값: 1024).
모델에 따라 적절한 길이 조정 필요 (예: BERT의 경우 512 제한).

num_workers:
데이터 로딩 및 전처리 시 사용할 worker 수.
None이면 기본값(os.cpu_count())이 사용됨.

In [12]:
args = Arguments(model_name_or_path="distilbert/distilbert-base-uncased",
                 torch_dtype='auto',
                 dataset_name="nyu-mll/glue",
                 dataset_config_name="mnli",
                 num_workers=1
                 )

In [37]:
lr = 2e-5
n_epochs = 10
batch_size = 64
max_len = 400

In [40]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',                 # 모델, log 등을 저장할 directory
    num_train_epochs= n_epochs,             # epoch 수
    per_device_train_batch_size=batch_size, # training data의 batch size
    per_device_eval_batch_size=batch_size,  # validation data의 batch size
    logging_strategy="epoch",               # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,                          # 학습을 진행하겠다는 의미
    do_eval=True,                           # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",                  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",                  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=lr,                       # optimizer에 사용할 learning rate
    load_best_model_at_end=True,            # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
    report_to = "wandb",
    push_to_hub=False
)


##데이터 업로드
Hugging Face Hub 에서 데이터 가져오기

In [15]:
raw_datasets = load_dataset(
    args.dataset_name,
    args.dataset_config_name
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [16]:
raw_datasets.shape

{'train': (392702, 4),
 'validation_matched': (9815, 4),
 'validation_mismatched': (9832, 4),
 'test_matched': (9796, 4),
 'test_mismatched': (9847, 4)}

In [17]:
print(raw_datasets['train'])

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})


In [18]:

train_df = pd.DataFrame(raw_datasets["train"])

In [62]:
train_dataset = raw_datasets['train']
df = pd.DataFrame(train_dataset)
print(df.head())

                                             premise  \
0  Conceptually cream skimming has two basic dime...   
1  you know during the season and i guess at at y...   
2  One of our number will carry out your instruct...   
3  How do you know? All this is their information...   
4  yeah i tell you what though if you go price so...   

                                          hypothesis  label  idx  
0  Product and geography are what make cream skim...      1    0  
1  You lose the things to the following level if ...      0    1  
2  A member of my team will execute your orders w...      0    2  
3                  This information belongs to them.      0    3  
4           The tennis shoes have a range of prices.      1    4  


In [63]:
print(train_df.describe(include="all").transpose())
print("------------")
print(train_df["label"].value_counts())

               count  unique         top freq      mean            std  min  \
premise       392702  128144  I blinked.   12       NaN            NaN  NaN   
hypothesis    392702  391501         n/a   40       NaN            NaN  NaN   
label       392702.0     NaN         NaN  NaN   1.00001       0.816499  0.0   
idx         392702.0     NaN         NaN  NaN  196350.5  113363.447042  0.0   

                 25%       50%        75%       max  
premise          NaN       NaN        NaN       NaN  
hypothesis       NaN       NaN        NaN       NaN  
label            0.0       1.0        2.0       2.0  
idx         98175.25  196350.5  294525.75  392701.0  
------------
label
2    130903
1    130900
0    130899
Name: count, dtype: int64


In [72]:
train_data, temp_data = train_test_split(train_df, train_size=0.05, test_size=0.03, random_state=42, stratify=train_df["label"])
val_data, test_data = train_test_split(temp_data, test_size=0.3333, random_state=42, stratify=temp_data["label"])

In [73]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

19635
7855
3927


In [74]:

from datasets import DatasetDict
dataset_dict = {
    'train': train_data,
    'val': val_data,
    'test': test_data,
}


In [75]:
print(len(dataset_dict['train']))
print(len(dataset_dict['val']))
print(len(dataset_dict['test']))

19635
7855
3927


## Tokenizer

In [76]:
model_name = "microsoft/deberta-base-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [77]:
from datasets import Dataset

#Hugging Face Dataset 포멧 전환
train_data = Dataset.from_pandas(train_data)
val_data = Dataset.from_pandas(val_data)
test_data = Dataset.from_pandas(test_data)

In [78]:
# tokenize and embedding
def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length")


In [79]:
#train, val, test 전처리
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/19635 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7855 [00:00<?, ? examples/s]

Map:   0%|          | 0/3927 [00:00<?, ? examples/s]

In [80]:
# "premise"、"hypothesis" 및 "idx" 열을 삭제하는 이유는 데이터 전처리 후 더 이상 필요하지 않기 때문입니다.
#모델은 분할된 특징(예: input_ids 및 attention_mask)과 레이블(label)만 사용하여 학습 및 평가를 수행합니다.
train_dt = train_data.remove_columns(["premise", "hypothesis", "idx"])
val_dt = val_data.remove_columns(["premise", "hypothesis", "idx"])
test_dt = test_data.remove_columns(["premise", "hypothesis", "idx"])

In [81]:
print(train_dt)

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 19635
})


In [82]:
#PyTorch tensor
train_dt.set_format("torch")
val_dt.set_format("torch")
test_dt.set_format("torch")

In [83]:


# 사용자 정의  DeBERTa 매개변수
config = DebertaConfig(
    hidden_size=64,             # layer의 기본 hidden dimension
    intermediate_size=64,       # FFN layer의 중간 hidden dimension
    num_attention_heads=4,      # Multi-head attention에서 사용하는 head 개수
    num_hidden_layers=2,        # 은닉층 수량
    num_labels=3                # 마지막에 예측해야 하는 분류 문제의 class 개수
)

model = AutoModelForSequenceClassification.from_config(config)


In [84]:
print(model.config)

DebertaConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 64,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 64,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-07,
  "legacy": true,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 4,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 64,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "transformers_version": "4.47.1",
  "type_vocab_size": 0,
  "vocab_size": 50265
}



In [ ]:
from huggingface_hub import login
login()

In [85]:
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [86]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dt,
    eval_dataset=val_dt,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

<ipython-input-86-911eb3369ec7>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [87]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.098700,1.098610,0.340802
2,1.098600,1.098601,0.333418
3,1.098600,1.098587,0.342839
4,1.098600,1.098560,0.354933
5,1.098500,1.098512,0.356334
6,1.098300,1.098431,0.355442
7,1.098200,1.098319,0.352260
8,1.097900,1.098203,0.353151
9,1.097600,1.098108,0.354806
10,1.097500,1.098075,0.354042


TrainOutput(global_step=3070, training_loss=1.098251720515447, metrics={'train_runtime': 1543.6763, 'train_samples_per_second': 127.196, 'train_steps_per_second': 1.989, 'total_flos': 6928326655266.0, 'train_loss': 1.098251720515447, 'epoch': 10.0})

In [88]:
wandb.finish()

eval/accuracy,█▂▁▂▂▃▃▂▂▂▂
eval/loss,▁██████████
eval/runtime,█▁▁▂▁▁▁▁▁▁▁
eval/samples_per_second,█▅▄▁▅▅▅▃▆▅▅
eval/steps_per_second,█▂▂▁▂▂▂▂▃▂▂
train/epoch,▁▁▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,██▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃
train/grad_norm,█▁▁▁▁▁▁▁▁▂▁
train/learning_rate,▅█▇▆▆▅▄▃▃▂▁
train/loss,▁███████▇▇▇
eval/accuracy,0.35404
